<h1> Groover's Algorithm </h1>
<br>
We want to search over a database o 4 elements and we will consider as the hidden element the state vector $|01\rangle$
<br>
The algorithm's circuit is as it follows:

![alt text](Grover.png "Title")

The algorithm for 2 qubits is exact so, the probability of the searched element will be 1

In [1]:
import cirq
import numpy as np
from qiskit import QuantumCircuit, execute, Aer
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (15,10)

<h2>CIRQ SCRIPT</h2>

In [2]:
def oracle(input_qubits, target_qubit, circuit, secret_element='01'):
    
    print(f"Element to be searched: {secret_element}")
    
    # Flip the qubits corresponding to the bits containing 0
    for i, bit in enumerate(secret_element):
        if int(bit) == 0:
            circuit.append(cirq.X(input_qubits[i]))
            
    # Do a Conditional NOT using all input qubits as control qubits
    circuit.append(cirq.TOFFOLI(*input_qubits, target_qubit))
    
    # Revert the input qubits to the state prior to Flipping
    for i, bit in enumerate(secret_element):
        if int(bit) == 0:
            circuit.append(cirq.X(input_qubits[i]))
            
    return circuit

def grovers_algorithm(num_qubits=2, copies=1000):
    
    # Define input and Target Qubit
    input_qubits = [cirq.LineQubit(i) for i in range(num_qubits)]
    target_qubit = cirq.LineQubit(num_qubits)
    
    # Define Quantum Circuit
    circuit = cirq.Circuit()
    
    # Create equal Superposition State
    circuit.append([cirq.H(input_qubits[i]) for i in range(num_qubits)])
    
    # Take target qubit to minus state |->
    circuit.append([cirq.X(target_qubit),cirq.H(target_qubit)])
    
    # Pass the qubit through the Oracle
    circuit = oracle(input_qubits, target_qubit, circuit)
    
    # Construct Grover operator.
    circuit.append(cirq.H.on_each(*input_qubits))
    circuit.append(cirq.X.on_each(*input_qubits))
    circuit.append(cirq.H.on(input_qubits[1]))
    circuit.append(cirq.CNOT(input_qubits[0],input_qubits[1]))
    circuit.append(cirq.H.on(input_qubits[1]))
    circuit.append(cirq.X.on_each(*input_qubits))
    circuit.append(cirq.H.on_each(*input_qubits))
    
    # Measure the result.
    circuit.append(cirq.measure(*input_qubits, key='Z'))
    print("Grover's algorithm follows")
    print(circuit)
    sim = cirq.Simulator()
    result = sim.run(circuit, repetitions=copies)
    out = result.histogram(key='Z')
    out_result = {}
    for k in out.keys():
        new_key = "{0:b}".format(k)
        if len(new_key) < num_qubits:
            new_key = (num_qubits - len(new_key))*'0' + new_key
        out_result[new_key] = out[k]
    print(out_result)

In [3]:
grovers_algorithm()

Element to be searched: 01
Grover's algorithm follows
0: ───H───X───@───X───H───X───@───X───H───────M('Z')───
              │               │               │
1: ───H───────@───H───X───H───X───H───X───H───M────────
              │
2: ───X───H───X────────────────────────────────────────
{'01': 1000}


<h2>QISKIT SCRIPT</h2>

In [4]:
def oracle_q(qc, secret_element='01'):
    
    print(f"Element to be searched: {secret_element}")
    
    # Flip the qubits corresponding to the bits containing 0
    for i, bit in enumerate(secret_element):
        if int(bit) == 0:
            qc.x(i)
            
    # Do a Conditional NOT using all input qubits as control qubits
    qc.ccx(0, 1, 2)
    
    # Revert the input qubits to the state prior to Flipping
    for i, bit in enumerate(secret_element):
        if int(bit) == 0:
            qc.x(i)
            
    return qc
    
def grovers_algorithm_q(num_qubits=2, copies=1000):
    
    # Define quantum circuit
    qc = QuantumCircuit(3,2)
    
    # Create equal Superposition State
    qc.h([0,1])
    qc.x(2)
    qc.h(2)
    
    # Pass the qubit through the Oracle
    qc = oracle_q(qc)
    
    # Construct Grover operator.
    qc.h([0,1])
    qc.x([0,1])
    qc.h(1)
    qc.cx(0, 1)
    qc.h(1)
    qc.x([0,1])
    qc.h([0,1])
    
    # Measure the result.
    qc.measure([0,1], [0,0])
    print("Grover's algorithm follows")
    print(qc)
    simulator = Aer.get_backend('qasm_simulator')
    job = execute(qc, simulator, shots=copies)
    res = job.result().get_counts(qc)
    print(res)

In [5]:
grovers_algorithm_q()

Element to be searched: 01
Grover's algorithm follows
     ┌───┐┌───┐     ┌───┐┌───┐┌───┐     ┌───┐┌───┐     ┌─┐   
q_0: ┤ H ├┤ X ├──■──┤ X ├┤ H ├┤ X ├──■──┤ X ├┤ H ├─────┤M├───
     ├───┤└───┘  │  ├───┤├───┤├───┤┌─┴─┐├───┤├───┤┌───┐└╥┘┌─┐
q_1: ┤ H ├───────■──┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├─╫─┤M├
     ├───┤┌───┐┌─┴─┐└───┘└───┘└───┘└───┘└───┘└───┘└───┘ ║ └╥┘
q_2: ┤ X ├┤ H ├┤ X ├────────────────────────────────────╫──╫─
     └───┘└───┘└───┘                                    ║  ║ 
c: 2/═══════════════════════════════════════════════════╩══╩═
                                                        0  0 
{'01': 1000}


As expected in both algorithm we found $|01\rangle$ as the only result.
<br>
As you can see the algorithm implemented in qiskit is cleaner.